### 0. Import libraries and import data

#### Questions:

**Q1:** Difference between flair tokenizer and Hugging Face tokenizer?\
**Q2:** Where do we have the POS and NER for questions?

#### Observations:

- There may be named entities composed by several words, but the predicted tags can be unmatched. (e.g.: B-LOC then I-MISC)
- Only ~3.6% (validation) / ~4.4% (train) of words have 3 or more predicted named entities for validation and .
- The tokenizer used by Flair extracts symbols like . or ) as separate words.
- TinyBERT tokenizer extracts symbols like . or ) as separate words and lowercases words.
- For unknown words, TinyBERT tokenizer splits the words in subwords and the subwords start with ##.
- TinyBERT tokenizer extarcts ' and s as 2 separated tokens and words with hyphens as 3 tokens, while Flair as one.
- Unrecognized tokens by the POS tagger are **$ '' , -LRB- -RRB- . : ``**

In [1]:
import re
import json
from collections import Counter

from flair.data import Sentence
from flair.models import SequenceTagger
from transformers import AutoTokenizer

In [2]:
input_file_path = "../../data/squad_data_train_pos_ner.json"
data = []

with open(input_file_path) as f:
    for line in f:
        data.append(json.loads(line))

### 1. Analyze POS tags

In [20]:
def analyze_sampled_context(data, example_index):
    context = ""
    pos_set = set()
    original_context = data[example_index]["context"]


    for word_pos in data[example_index]["POS_context"]:
        word, pos = list(word_pos.values())[0]
        context += word
        context += " "
        pos_set.add(pos)

    print(f"Reconstructed context:\n{context}")
    print(f"\nOriginal context:\n{original_context}")
    print(f"\nPOS set: {pos_set}")

    return context

In [21]:
reconstructed_context = analyze_sampled_context(data, 0)

Reconstructed context:
Architecturally , the school has a Catholic character . Atop the Main Building 's gold dome is a golden statue of the Virgin Mary . Immediately in front of the Main Building and facing it , is a copper statue of Christ with arms upraised with the legend " Venite Ad Me Omnes " . Next to the Main Building is the Basilica of the Sacred Heart . Immediately behind the basilica is the Grotto , a Marian place of prayer and reflection . It is a replica of the grotto at Lourdes , France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858 . At the end of the main drive ( and in a direct line that connects through 3 statues and the Gold Dome ) , is a simple , modern stone statue of Mary . 

Original context:
Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the l

In [24]:
def get_all_pos_tags(data):
    pos_set = set()

    for example_index in range(len(data)):
        for word_pos in data[example_index]["POS_context"]:
            _, pos = list(word_pos.values())[0]
            pos_set.add(pos)

    all_pos = sorted(list(pos_set))

    print(f"\nPOS set: {all_pos}")

In [25]:
get_all_pos_tags(data)


POS set: ['$', "''", ',', '-LRB-', '-RRB-', '.', ':', 'ADD', 'AFX', 'CC', 'CD', 'DT', 'EX', 'FW', 'HYPH', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NFP', 'NN', 'NNP', 'NNPS', 'NNS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB', 'XX', '``']


In [36]:
data[0]["POS_context"]

[{'0': ['Architecturally', 'RB']},
 {'1': [',', ',']},
 {'2': ['the', 'DT']},
 {'3': ['school', 'NN']},
 {'4': ['has', 'VBZ']},
 {'5': ['a', 'DT']},
 {'6': ['Catholic', 'JJ']},
 {'7': ['character', 'NN']},
 {'8': ['.', ',']},
 {'9': ['Atop', 'IN']},
 {'10': ['the', 'DT']},
 {'11': ['Main', 'NNP']},
 {'12': ['Building', 'NNP']},
 {'13': ["'s", 'POS']},
 {'14': ['gold', 'JJ']},
 {'15': ['dome', 'NN']},
 {'16': ['is', 'VBZ']},
 {'17': ['a', 'DT']},
 {'18': ['golden', 'JJ']},
 {'19': ['statue', 'NN']},
 {'20': ['of', 'IN']},
 {'21': ['the', 'DT']},
 {'22': ['Virgin', 'NNP']},
 {'23': ['Mary', 'NNP']},
 {'24': ['.', ',']},
 {'25': ['Immediately', 'RB']},
 {'26': ['in', 'IN']},
 {'27': ['front', 'NN']},
 {'28': ['of', 'IN']},
 {'29': ['the', 'DT']},
 {'30': ['Main', 'NNP']},
 {'31': ['Building', 'NNP']},
 {'32': ['and', 'CC']},
 {'33': ['facing', 'VBG']},
 {'34': ['it', 'PRP']},
 {'35': [',', ',']},
 {'36': ['is', 'VBZ']},
 {'37': ['a', 'DT']},
 {'38': ['copper', 'NN']},
 {'39': ['statue', '

In [8]:
def check_square_brackets(data):
    count = 0 

    for example_index in range(len(data)):
        if "[" in data[example_index]["context"] or "]" in data[example_index]["context"]:
            count += 1

    print(f"Number of examples that contain a square bracket: {count}")

In [9]:
check_square_brackets(data)

Number of examples that contain a square bracket: 7042


### 2. Analyze named entities

In [6]:
def count_ner_number(data):
    ner_number = []

    for example in data:
        for word_ners in example["NER_context"]:
            ner_number.append(len(list(word_ners.values())[0]))

    words_no = len(ner_number)
    print("Percentage of words that have a specific number of named entities:")

    for number in range(1, len(set(ner_number))+ 1):
        print(f"{number} - {round(Counter(ner_number)[number] / words_no * 100, 2)}%")

In [7]:
count_ner_number(data)

Percentage of words that have a specific number of named entities:
1 - 76.5%
2 - 19.07%
3 - 3.87%
4 - 0.52%
5 - 0.03%
6 - 0.0%


In [68]:
data[0]["NER_context"]

[{'0': [['Super', 'B-MISC', '0.9410769']]},
 {'1': [['Bowl', 'E-MISC', '0.98900163']]},
 {'5': [['American', 'S-MISC', '0.9956988']]},
 {'14': [['National', 'B-MISC', '0.5022688'],
   ['National', 'B-LOC', '0.24889919'],
   ['National', 'B-ORG', '0.19581686']]},
 {'15': [['Football', 'I-MISC', '0.6923045'],
   ['Football', 'I-ORG', '0.18027079'],
   ['Football', 'I-LOC', '0.12511595']]},
 {'16': [['League', 'E-ORG', '0.60113037'],
   ['League', 'E-MISC', '0.38968554']]},
 {'18': [['NFL', 'S-ORG', '0.99872464']]},
 {'26': [['American', 'B-MISC', '0.7453571'],
   ['American', 'B-ORG', '0.13586394']]},
 {'27': [['Football', 'I-MISC', '0.9433532']]},
 {'28': [['Conference', 'E-MISC', '0.74766535'],
   ['Conference', 'I-MISC', '0.13813968'],
   ['Conference', 'E-ORG', '0.11040713']]},
 {'30': [['AFC', 'S-ORG', '0.9421894']]},
 {'33': [['Denver', 'B-ORG', '0.9724425']]},
 {'34': [['Broncos', 'E-ORG', '0.99889153']]},
 {'37': [['National', 'B-MISC', '0.53137046'],
   ['National', 'B-ORG', '0.

In [20]:
tagger = SequenceTagger.load("ner-fast")

2023-10-20 19:14:53,106 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [28]:
sentence = Sentence('Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levis Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.')
tagger.predict(sentence)
print(sentence)

Sentence[147]: "Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levis Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50." → ["Super Bowl"/MISC, "American"/MISC, "National Football League"/ORG, "NFL"/ORG, "American Football Conference"/MISC, "AFC"/ORG, "Denver Broncos"/ORG, "National Football Conference"/ORG, "NFC"/ORG, "Carolina

### 3. Analyze TinyBERT tokenizer

In [107]:
tokenizer = AutoTokenizer.from_pretrained("../../models/fine_tuned_tiny_bert_8_0.0003_tokenizer/")

input_ids = tokenizer.encode("", 'Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.')

tokens = tokenizer.convert_ids_to_tokens(input_ids)
tokens

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


['[CLS]',
 '[SEP]',
 'super',
 'bowl',
 '50',
 'was',
 'an',
 'american',
 'football',
 'game',
 'to',
 'determine',
 'the',
 'champion',
 'of',
 'the',
 'national',
 'football',
 'league',
 '(',
 'nfl',
 ')',
 'for',
 'the',
 '2015',
 'season',
 '.',
 'the',
 'american',
 'football',
 'conference',
 '(',
 'afc',
 ')',
 'champion',
 'denver',
 'broncos',
 'defeated',
 'the',
 'national',
 'football',
 'conference',
 '(',
 'nfc',
 ')',
 'champion',
 'carolina',
 'panthers',
 '24',
 '–',
 '10',
 'to',
 'earn',
 'their',
 'third',
 'super',
 'bowl',
 'title',
 '.',
 'the',
 'game',
 'was',
 'played',
 'on',
 'february',
 '7',
 ',',
 '2016',
 ',',
 'at',
 'levi',
 "'",
 's',
 'stadium',
 'in',
 'the',
 'san',
 'francisco',
 'bay',
 'area',
 'at',
 'santa',
 'clara',
 ',',
 'california',
 '.',
 'as',
 'this',
 'was',
 'the',
 '50th',
 'super',
 'bowl',
 ',',
 'the',
 'league',
 'emphasized',
 'the',
 '"',
 'golden',
 'anniversary',
 '"',
 'with',
 'various',
 'gold',
 '-',
 'themed',
 'init

In [29]:
tokenizer = AutoTokenizer.from_pretrained("../../models/fine_tuned_tiny_bert_8_0.0003_tokenizer/")
tokenizer.add_tokens(["[SYM]"])

input_ids = tokenizer.encode("", 'Super Bowl [SYM], 50, was an American football')

tokens = tokenizer.convert_ids_to_tokens(input_ids)
tokens

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


['[CLS]',
 '[SEP]',
 'super',
 'bowl',
 '[SYM]',
 ',',
 '50',
 ',',
 'was',
 'an',
 'american',
 'football',
 '[SEP]']

### 4. Compare Flair and TinyBERT tokenizers

In [104]:
reconstructed_context_tokenizer = " ".join(tokens).replace(" ##", "").replace("[SEP]", "").replace("[CLS]", "").replace("  ", " ").strip(" ")
print(f"Reconstructed context processed by TinyBERT tokenizer:\n{reconstructed_context_tokenizer}")
print(f"\nReconstructed context processed by Flair:\n{reconstructed_context}")

Reconstructed context processed by TinyBERT tokenizer:
super bowl 50 was an american football game to determine the champion of the national football league ( nfl ) for the 2015 season . the american football conference ( afc ) champion denver broncos defeated the national football conference ( nfc ) champion carolina panthers 24 – 10 to earn their third super bowl title . the game was played on february 7 , 2016 , at levi ' s stadium in the san francisco bay area at santa clara , california . as this was the 50th super bowl , the league emphasized the " golden anniversary " with various gold - themed initiatives , as well as temporarily suspending the tradition of naming each super bowl game with roman numerals ( under which the game would have been known as " super bowl l " ) , so that the logo could prominently feature the arabic numerals 50 .

Reconstructed context processed by Flair:
Super Bowl 50 was an American football game to determine the champion of the National Football Lea

In [106]:
list(zip(reconstructed_context_tokenizer.split(" "), reconstructed_context.split(" ")))

[('super', 'Super'),
 ('bowl', 'Bowl'),
 ('50', '50'),
 ('was', 'was'),
 ('an', 'an'),
 ('american', 'American'),
 ('football', 'football'),
 ('game', 'game'),
 ('to', 'to'),
 ('determine', 'determine'),
 ('the', 'the'),
 ('champion', 'champion'),
 ('of', 'of'),
 ('the', 'the'),
 ('national', 'National'),
 ('football', 'Football'),
 ('league', 'League'),
 ('(', '('),
 ('nfl', 'NFL'),
 (')', ')'),
 ('for', 'for'),
 ('the', 'the'),
 ('2015', '2015'),
 ('season', 'season'),
 ('.', '.'),
 ('the', 'The'),
 ('american', 'American'),
 ('football', 'Football'),
 ('conference', 'Conference'),
 ('(', '('),
 ('afc', 'AFC'),
 (')', ')'),
 ('champion', 'champion'),
 ('denver', 'Denver'),
 ('broncos', 'Broncos'),
 ('defeated', 'defeated'),
 ('the', 'the'),
 ('national', 'National'),
 ('football', 'Football'),
 ('conference', 'Conference'),
 ('(', '('),
 ('nfc', 'NFC'),
 (')', ')'),
 ('champion', 'champion'),
 ('carolina', 'Carolina'),
 ('panthers', 'Panthers'),
 ('24', '24'),
 ('–', '–'),
 ('10', '1

### 5. Check answers

In [14]:
for index, line in enumerate(data):
    if index > 2000:
        break
    
    answer_text = line["answers"]["text"][0]
    answer_start = line["answers"]["answer_start"]

    try:
        found_answer_start = [match.start() for match in re.finditer(answer_text, line["context"])]

        print(index, answer_start, found_answer_start)
    except:
        pass


0 [515] [515]
1 [188] [188]
2 [279] [59, 152, 279]
3 [381] [381]
4 [92] [92]
5 [248] [248]
6 [441] [298, 441]
7 [598] [598, 778, 947, 1226]
8 [126] [126, 1253]
9 [908] [908]
10 [119] [119]
11 [145] [145, 521, 649]
12 [234] [234]
13 [356] [356]
14 [675] [675]
15 [487] [487]
16 [46] [46]
17 [126] [126]
18 [271] [271, 548]
19 [155] [155]
20 [496] [496]
21 [68] [68]
22 [155] [155]
23 [647] [647]
24 [358] [358]
25 [624] [624]
26 [1163] [1163]
27 [92] [92]
28 [757] [757]
29 [4] [4]
30 [466] [466]
31 [303] [303]
32 [377] [377]
33 [360] [360]
34 [136] [136]
35 [145] [145]
36 [188] [188]
37 [344] [344]
38 [394] [394]
39 [109] [109, 481]
40 [138] [138]
41 [213] [213]
42 [488] [488]
43 [618] [618]
44 [32] [32]
45 [362] [33, 362, 1223]
46 [565] [565]
47 [155] [155]
48 [918] [918]
49 [0] [0]
50 [353] [353]
51 [406] [406]
52 [638] [638]
53 [85] [85]
54 [3] [3]
55 [136] [136]
56 [123] [123]
57 [222] [222]
58 [49] [49]
59 [0] [0]
60 [963] [963]
61 [1049] [1049]
62 [1099] [1099]
63 [86] [86]
64 [0] [0]

### 6. Check space sequences

In [47]:
for space_no in range(2, 100):
    found = False

    for line in data:
        index = line["context"].find(space_no * " ")
        
        
        if index != -1:
            found = True

    if found:
        print(f"Found {space_no} spaces!")            

Found 2 spaces!
Found 3 spaces!
Found 4 spaces!


In [38]:
"ana ".find("  ")

-1